In [ ]:
import matplotlib,numpy,glob
import common_functions as cf
import matplotlib.pyplot as plt
from astropy.table import Table

In [ ]:
HCGs = Table.read('../output/HCGs.vo', format='votable')
HCGs.add_index('HCG')

HCG_mems = Table.read('../output/HCG_members.vo', format='votable')

HCGs['logMHI_gals'] = numpy.zeros(len(HCGs))+numpy.nan
HCGs['logMHI_exfs'] = numpy.zeros(len(HCGs))+numpy.nan

HCG_mems['logMHI'] = numpy.zeros(len(HCG_mems))+numpy.nan

In [ ]:
for HCG in list(HCGs['HCG']):
    file_list = glob.glob(f'SoFiA_masks/HCG{HCG}/separated_features/*.fits')
    if len(file_list) > 0:
        print(f'Opening the following files for HCG{HCG}:')
        print('\t'+'\n\t'.join(file_list))

        members = list(HCG_mems['name'][HCG_mems['HCG'] == HCG])

        for pb_cubelet in file_list:

            cube,cube_ra,cube_dec,cube_vel,bmaj,bmin,pa,beam_factor,cube_dx,cube_dy,cube_dv = cf.read_fitscube(pb_cubelet,True,True)
            
            #Catch the error that SlicerAstro sometimes writes an incorrect value in pixel (0,0,0)
            if abs(cube[0,0,0]) > 1.:
                cube[0,0,0] = 0.

            mhi = numpy.log10(numpy.nansum(cube.flatten()/beam_factor)*abs(cube_dv)*235600.*HCGs.loc[HCG]['dist']**2.)

            #Identify what object is classified as
            group_feature = False
            feat_code = ['br','ex','tl','cl']
            for feature in feat_code:
                if feature in pb_cubelet:
                    inx = pb_cubelet.index(feature)
                    print(f'HCG{HCG}'+pb_cubelet[inx:inx+len(feature)+1]+' is a non-galactic feature.')
                    print(f'HCG{HCG}'+pb_cubelet[inx:inx+len(feature)+1]+f' has a log HI mass of:{mhi}')
                    group_feature = True
                    if numpy.isfinite(HCGs.loc[HCG]['logMHI_exfs']):
                        HCGs.loc[HCG]['logMHI_exfs'] = numpy.log10(10.**mhi + 10.**HCGs.loc[HCG]['logMHI_exfs'])
                    else:
                        HCGs.loc[HCG]['logMHI_exfs'] = mhi
                    break
                        
            group_member = False
            if not group_feature:
                for mem in members:
                    if mem+'.fits' in pb_cubelet:
                        print(f'{mem} is a member of HCG{HCG}.')
                        print(f'{mem} has a log HI mass of:{mhi}')
                        group_member = True
                        HCG_mems[HCG_mems['name']==mem]['logMHI'] = mhi
                        if numpy.isfinite(HCGs.loc[HCG]['logMHI_gals']):
                            HCGs.loc[HCG]['logMHI_gals'] = numpy.log10(10.**mhi + 10.**HCGs.loc[HCG]['logMHI_gals'])
                        else:
                            HCGs.loc[HCG]['logMHI_gals'] = mhi
                        break

            if not group_feature and not group_member:
                inx1 = pb_cubelet.index('features/')+9
                inx2 = pb_cubelet.index('.fits')
                print(pb_cubelet[inx1:inx2]+f' is not part of HCG{HCG}.')
                print(pb_cubelet[inx1:inx2]+f' has a log HI mass of:{mhi}')
    else:
        print(f'No files found for HCG{HCG}.')

    print('\n\n')

In [ ]:
HCGs

In [ ]:
HCGs['HIdef'] = HCGs['logMHI_pred'] - numpy.log10(numpy.where(numpy.isfinite(HCGs['logMHI_gals']),10.**HCGs['logMHI_gals'],0.) + numpy.where(numpy.isfinite(HCGs['logMHI_exfs']),10.**HCGs['logMHI_exfs'],0.))
HCGs['ex_frac'] = (numpy.where(numpy.isfinite(HCGs['logMHI_exfs']),10.**HCGs['logMHI_exfs'],0.))/(numpy.where(numpy.isfinite(HCGs['logMHI_gals']),10.**HCGs['logMHI_gals'],0.) + numpy.where(numpy.isfinite(HCGs['logMHI_exfs']),10.**HCGs['logMHI_exfs'],0.))

In [ ]:
HCGs

In [ ]:
plt.scatter(HCGs['HIdef'],HCGs['ex_frac'])